## The purpose of this notebook is just to clean the data for further analysis

### Importing Libraries

In [36]:
## Importing the required libraries
import pandas as pd 
import numpy as np
import re
import matplotlib.pyplot as plt
%matplotlib inline

### Data Gathering

In [37]:
## Loading the data into Pandas dataframes

# application data
applications = pd.read_csv("/content/Wuzzuf_Applications_Sample.csv")

# posts data
posts = pd.read_csv("/content/Wuzzuf_Job_Posts_Sample.csv")

# exploring the top 5 rows

display(applications.head())
display(posts.head())

,id,user_id,job_id,app_date
0,ba7b8f17,846d013c,516e4ed,2014-01-01 07:27:52
1,30e1ae86,9d5e32c5,516e4ed,2014-01-01 08:20:42
2,d829a6b7,eb26a291,516e4ed,2014-01-01 10:30:37
3,3f985f37,7b5e68a8,516e4ed,2014-01-01 10:55:07
4,27e1695,76fa79b1,516e4ed,2014-01-01 11:00:26


,id,city,job_title,job_category1,job_category2,job_category3,job_industry1,job_industry2,job_industry3,salary_minimum,salary_maximum,num_vacancies,career_level,experience_years,post_date,views,job_description,job_requirements,payment_period,currency
0,516e4ed,Ciro,Sales & Marketing Agent,Sales/Retail/Business Development,Marketing,Select,Telecommunications Services,Select,Select,2000,3500,8,Entry Level,0-1,2014-01-01 06:01:41,2602,<p><strong>Qualifications</strong>:<br /><br /...,NaN,Per Month,Egyptian Pound
1,a361ef59,Cairo,German Training Coordinator,Customer Service/Support,Administration,Human Resources,Translation and Localization,Business Services - Other,Education,1000,5000,8,Entry Level,0-2,2014-01-01 20:01:18,2213,<p>&bull;Placing jobs' ads on various websites...,NaN,Per Month,Egyptian Pound
2,7226ce78,Cairo,Junior Software Developer,IT/Software Development,Select,Select,Computer Software,Select,Select,2000,2500,1,Entry Level,2,2014-01-02 11:01:03,2940,"<span style=""text-decoration: underline;""><str...",NaN,Per Month,Egyptian Pound
3,f4b2bcd6,Cairo,Application Support Engineer,IT/Software Development,Select,Select,Telecommunications Services,Select,Select,2000,3500,1,Entry Level,1-2,2014-01-02 12:01:23,2042,"<strong><span style=""text-decoration: underlin...",NaN,Per Month,Egyptian Pound
4,3fee6f73,Alexandria,Electrical Maintenance Engineer,Engineering,Select,Select,Food and Beverage Production,Select,Select,5000,8000,1,Experienced (Non-Manager),1-3,2014-01-21 13:45:56,5684,Job Title: Electrical Maintenance Engineer<br ...,NaN,Per Month,Egyptian Pound


In [38]:
# make a copy of our data to clean it away from the source
applications_df = applications
# The posts data
job_posts_df = posts

### Data Cleaning

#### 1) Applications Dataframe

In [39]:
# fixing the application date
applications_df.app_date = pd.to_datetime(applications_df.app_date)

# test
applications_df.dtypes

id                  object
user_id             object
job_id              object
app_date    datetime64[ns]
dtype: object

In [40]:
# set the index for app_date
applications_df.index = applications_df['app_date']

# remove the app_date column
applications_df.drop(columns='app_date',axis= 1,inplace=True)

applications_df["count"] = 1

# Test
applications_df.head()

,id,user_id,job_id,count
app_date,,,,
2014-01-01 07:27:52,ba7b8f17,846d013c,516e4ed,1
2014-01-01 08:20:42,30e1ae86,9d5e32c5,516e4ed,1
2014-01-01 10:30:37,d829a6b7,eb26a291,516e4ed,1
2014-01-01 10:55:07,3f985f37,7b5e68a8,516e4ed,1
2014-01-01 11:00:26,27e1695,76fa79b1,516e4ed,1


#### 2) Posts Dataframe

In [41]:
# explore more info about the job-posts data
job_posts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21850 entries, 0 to 21849
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                21850 non-null  object
 1   city              21850 non-null  object
 2   job_title         21850 non-null  object
 3   job_category1     21850 non-null  object
 4   job_category2     21850 non-null  object
 5   job_category3     21850 non-null  object
 6   job_industry1     21850 non-null  object
 7   job_industry2     21850 non-null  object
 8   job_industry3     21850 non-null  object
 9   salary_minimum    21850 non-null  int64 
 10  salary_maximum    21850 non-null  int64 
 11  num_vacancies     21850 non-null  int64 
 12  career_level      21850 non-null  object
 13  experience_years  21850 non-null  object
 14  post_date         21850 non-null  object
 15  views             21850 non-null  int64 
 16  job_description   21576 non-null  object
 17  job_requirem

##### Columns cleaning:
> - columns to be dropped  -->  'job_category2','job_category3','job_industry2','job_industry3'
- fixing the post_date column -->  datetime
- create new columns       -->  day and month for more insights

In [42]:
# drop the unneeded columns 
columns_drop = ['job_category2','job_category3','job_industry2','job_industry3']
job_posts_df.drop(columns=columns_drop, axis=1, inplace=True)
           

# rename the columns
job_posts_df.rename(columns={'job_category1': 'job_category', 'job_industry1': 'job_industry'}, inplace=True)

# convert post_date to datetime 
job_posts_df.post_date = pd.to_datetime(job_posts_df['post_date'])

# extract month name
job_posts_df['month'] = job_posts_df.post_date.apply(lambda x: x.month_name())

# extract  name of the day
job_posts_df['day'] = job_posts_df.post_date.apply(lambda x: x.day_name())


# check for data sample
job_posts_df.sample(5)

,id,city,job_title,job_category,job_industry,salary_minimum,salary_maximum,num_vacancies,career_level,experience_years,post_date,views,job_description,job_requirements,payment_period,currency,month,day
6239,4ef81b0b,Cairo,Junior SAP ABAP Consultant,IT/Software Development,Business Services - Other,3000,4000,1,Experienced (Non-Manager),2+,2015-01-11 14:01:35,1543,Junior SAP ABAP Consultant,<ul>\r\n<li>ABAP Workbench navigation.</li>\r\...,Per Month,Egyptian Pound,January,Sunday
16867,681b0c38,Cairo,Java Developer,IT/Software Development,Information Technology Services,4000,5000,1,Experienced (Non-Manager),2+,2016-01-11 18:01:27,10,"<ul>\r\n<li>BS/MS degree in Computer Science, ...",<ul>\r\n<li>Minimum 2 years working experience...,Per Month,Egyptian Pound,January,Monday
19165,4bb8903e,Cairo,Technical Support Agent,IT/Software Development,Computer Software,1200,1500,1,Entry Level,0-2,2016-03-14 10:03:30,596,<ul>\r\n<li>You will be responsible for provid...,<ul>\r\n<li>Fresh graduates are welcome to app...,Per Month,Egyptian Pound,March,Monday
3499,df345ed2,Giza,PR Manager,Marketing,Education,1000,5000,1,Entry Level,1+,2014-08-28 21:08:46,4,<ul>\r\n<li>We need someone who's great at Mar...,NaN,Per Month,Egyptian Pound,August,Thursday
10619,1cbc624e,Cairo,Web Designer,Creative/Design,Advertising and PR Services,3000,5000,2,Entry Level,1+,2015-06-23 12:06:28,1374,<ul>\r\n<li>Responsible for the design and dev...,<ul>\r\n<li>Good command of English language s...,Per Month,Egyptian Pound,June,Tuesday


##### Missing values and duplicates:

In [43]:
# check for null values
job_posts_df.isnull().sum()

id                     0
city                   0
job_title              0
job_category           0
job_industry           0
salary_minimum         0
salary_maximum         0
num_vacancies          0
career_level           0
experience_years       0
post_date              0
views                  0
job_description      274
job_requirements    2633
payment_period         5
currency               5
month                  0
day                    0
dtype: int64

So, we have missing values in columns:  'job_description' and 'job-requirement' we can pass it by now; as we are not interested in these columns for the puropse of our analysis, so we can check first for the currency column.

In [44]:
# check for the null values in currency column
job_posts_df.loc[job_posts_df.currency.isnull()]

,id,city,job_title,job_category,job_industry,salary_minimum,salary_maximum,num_vacancies,career_level,experience_years,post_date,views,job_description,job_requirements,payment_period,currency,month,day
11264,671459f6,الجيزة,مطلوب موظفين كول سنتر و مدخلين بيانات للعمل بك...,Customer Service/Support,Marketing and Advertising,1000,3000,1,Student,0/1,2015-07-24 15:07:18,2,<strong>لا يشترط خبره و لا يشترط مؤهل</strong>...,<ol>\r\n<li><strong>مهارات تواصل جيدة</strong>...,NaN,NaN,July,Friday
11267,fb9bd9dc,الجيزة,مطلوب مندوب مبيعات للعمل بكبرى الشركات,Marketing,Agriculture/Forestry/Fishing/Farming,1000,2500,1,Student,0-1,2015-07-24 16:07:10,1,لا يشترط خبرة و لا يشترط مؤهل<br />السن:18/40<...,القدرة علي الاقناع<br />حسن المظهر,NaN,NaN,July,Friday
11268,30c70a79,الجيزة,مطلوب موظفين كول سنتر و مدخلين بيانات للعمل بك...,Customer Service/Support,Outsourcing/Offshoring,1000,3000,1,Student,0-1,2015-07-24 16:07:36,0,لا يشترط مؤهل او خبرة سابقة<br />السن المطلوب:...,مهارات تواصل جيدة<br />القدرة علي تحديد مشكلة ...,NaN,NaN,July,Friday
19501,4ae1b1df,Cairo,Architect,Engineering,Architectural and Design Services,3000,4500,1,Experienced (Non-Manager),3,2016-03-21 16:03:39,0,NaN,NaN,NaN,NaN,March,Monday
19502,cc75c371,Cairo,Urban Planner,Engineering,Architectural and Design Services,3000,4500,1,Entry Level,3,2016-03-21 16:03:56,0,NaN,NaN,NaN,NaN,March,Monday


So, these are missed and we can fill it with Egyptian Pound as it seems local job_posts

In [45]:
# fix the null values and converts it to Egyptian pound
job_posts_df.currency.fillna('Egyptian Pound', inplace = True)

# check
job_posts_df.currency.isnull().sum()

0

##### Experience_years columns:
> - This column contains messy experience years ranges
- unify the experience range for better analysis (Data Uniquness)
- create new column experience_range

In [46]:
# check for the number of unique values in experience_years column
job_posts_df.experience_years.nunique()

461

In [47]:
# check for the experience years
job_posts_df.experience_years.unique()[::-10]

array(['3 :5', '6 - 8 ', '4  to  6', '3--5', '3-7 ', '2 at least',
       '3year', '1 - 2 years',
       '3-5 Years of experience in Cement Industry', '5 ', '5-6 ', '1- 5',
       '<8', '0 or 1 year', '7  to 9 ', '6+ ', '0 - 5', '6-12', '9-15',
       '5-8 ', ' 2+', '0,1', '4 years', '3 سنوات على الاقل', '0 - 1',
       '1-7', '8 -10', '0 - 3', '5 -7 ', '1- 3', '4-5 ', '2:4', '0 -2',
       '1~2', '1-5 ', '2-3 ', '2:3', '7 - 15', '1-3 ', '20-25', '1-4 ',
       '0-1 ', '7-10', '0 , 1 or 2', '3-5', '6', '0-1'], dtype=object)

- as we notice here, there is a messy categories we are going to fix it so expereience range will be as:
  Fresh, <5,5-10,10-15,15-20,20-25,>25

In [48]:
# fix the experience year column
l = job_posts_df.experience_years.values
range_exps =[]
for i in l:
    pattern =re.findall('\\b\\d+\\b', i)
    if len(pattern) == 0:
        #min_exp = min([int(s) for s in i if s.isdigit()])
        max_exp = max([int(s) for s in i if s.isdigit()])
    else:
        #min_exp = min([int(s) for s in pattern])
        max_exp = max([int(s) for s in pattern])
    # check for the + sign
    if '+' in i:
        max_exp += 3
    # add to our cleaned list
    if max_exp == 0 :
        range_exp = 'Fresh'
    elif 0 < max_exp < 5:
        range_exp = 'below 5'
    elif 5 <= max_exp < 10:
        range_exp = '5-10'
    elif 10 <= max_exp < 15:
        range_exp = '10-15'
    elif 15 <= max_exp < 20:
        range_exp = '15-20'
    elif 20 <= max_exp <= 25:
        range_exp = '20-25'
    else:
        range_exp = 'above 25'
    range_exps.append(range_exp)

In [49]:
# check the no. of values
len(range_exps) == job_posts_df.shape[0]

True

In [50]:
# so now we can add the cleaned range to our dataset
job_posts_df['experience_range'] = range_exps

In [51]:
# experience range new values
job_posts_df.experience_range.unique()

array(['below 5', '5-10', '10-15', '15-20', 'Fresh', '20-25', 'above 25'],
      dtype=object)

In [52]:
# further check for some samples from the dataset
job_posts_df.sample(5,random_state=1)[['experience_years','experience_range']]

,experience_years,experience_range
15815,1-3,below 5
9729,0-1,below 5
9398,1+,below 5
33,3-7,5-10
12291,2-5,5-10


In [53]:
# now we can drop the uncleaned column of experience_years
job_posts_df.drop(columns= 'experience_years',inplace=True)

# viewing a random sample of so far-cleaned 
job_posts_df.sample(5, random_state=1)

,id,city,job_title,job_category,job_industry,salary_minimum,salary_maximum,num_vacancies,career_level,post_date,views,job_description,job_requirements,payment_period,currency,month,day,experience_range
15815,45142a1e,Cairo,Telesales Agent - Software/ hardware Products,Sales/Retail/Business Development,Computer/IT Services,1200,1500,3,Entry Level,2015-12-13 16:12:10,839,<strong>About the Job and Duties :</strong><br...,<ul>\r\n<li>Bachelor Degree: Commerce or equiv...,Per Month,Egyptian Pound,December,Sunday,below 5
9729,7d5476e3,Cairo,International Customer Service Representative ...,Customer Service/Support,Telecommunications Services,2200,4600,40,Entry Level,2015-05-20 15:05:32,2314,<ul>\r\n<li>Handle German Customers inquires a...,<ul>\r\n<li>Fluent German</li>\r\n<li>Excellen...,Per Month,Egyptian Pound,May,Wednesday,below 5
9398,77796efd,Giza,Call Center Agent,Sales/Retail/Business Development,Marketing and Advertising,2000,2500,2,Entry Level,2015-05-21 14:39:59,1887,<ul>\r\n<li>Advises present or prospective cus...,<ul>\r\n<li>1 years&rsquo; experience in the t...,Per Month,Egyptian Pound,May,Thursday,below 5
33,77d749b9,Cairo,Senior PHP Developer,IT/Software Development,Computer Software,3500,7500,5,Experienced (Non-Manager),2014-01-05 19:01:23,1055,"<p><span style=""text-decoration: underline;""><...","<p>1. Solid commercial knowledge of PHP, XHTML...",Per Month,Egyptian Pound,January,Sunday,5-10
12291,cb81f16,Cairo,JAVA SE & Android Trainer /Instructor,Education/Training,Education,1000,50000,2,Experienced (Non-Manager),2015-08-27 10:08:36,1364,<ul>\r\n<li>Responsible for training individua...,<ul>\r\n<li>Have a bachelor's degree</li>\r\n<...,Per Month,Egyptian Pound,August,Thursday,5-10


##### Cleaning Job description and Job requirements columns

Now these two columns contains HTML character and you will try to clean it
by the the help of stackoverflow I found this answer [here](https://stackoverflow.com/a/925630/10708662)

In [54]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed).strip().replace('\r\n',' ').replace('\xa0','') # more cleaning for the extra \n\r or \xa0

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [55]:
# fix the job_description 
job_posts_df.job_description = job_posts_df.job_description.apply(lambda x: strip_tags(x) if isinstance(x,str) else x)
job_posts_df.job_description[0]

'Qualifications: • Graduates and Undergraduates are encouraged to apply. • Telesales experience 6 Months at least.  • Proficiency in using MS Office application ”Word, Excel” • Strong verbal communication and negotiation skills • Demonstrates a positive, enthusiastic, friendly and professional attitude  English Skills: Fluent    Job Details and Benefits: • Available Start Date: In 2 Weeks • Working Hours: 8.5 hours • Location: Maadi, Cairo  Job Package: . Basic salary 2000 EGP + Commission when Target is Acheived'

In [56]:
# fix the job_requirements type(x) == str
job_posts_df.job_requirements = job_posts_df.job_requirements.apply(lambda x: strip_tags(x) if isinstance(x,str) else x)
job_posts_df.job_description[0]

'Qualifications: • Graduates and Undergraduates are encouraged to apply. • Telesales experience 6 Months at least.  • Proficiency in using MS Office application ”Word, Excel” • Strong verbal communication and negotiation skills • Demonstrates a positive, enthusiastic, friendly and professional attitude  English Skills: Fluent    Job Details and Benefits: • Available Start Date: In 2 Weeks • Working Hours: 8.5 hours • Location: Maadi, Cairo  Job Package: . Basic salary 2000 EGP + Commission when Target is Acheived'

In [57]:
# check for our work
job_posts_df.sample(5,random_state=1)[['job_description','job_requirements']]

,job_description,job_requirements
15815,About the Job and Duties : Telemarketing\Tele...,Bachelor Degree: Commerce or equivalent Experi...
9729,Handle German Customers inquires and complaint...,Fluent German Excellent communication & proble...
9398,Advises present or prospective customers by an...,1 years’ experience in the telesales / telemar...
33,Desired Experience PHP Development Experience...,"1. Solid commercial knowledge of PHP, XHTML, C..."
12291,Responsible for training individuals. Develop ...,Have a bachelor's degree Very good English lan...


##### Clean city column
> - contains misspelled cities names.
- Could be more elegant if grouped as Cairo, Alexandria , Delta , upper Egypt and all over country

In [58]:
# check for the number of unique records
job_posts_df.city.nunique()

478

In [59]:
# defining a function for stripping city names
def strip_city_name(x):
    if x[-1] ==',':
        x = x[:-1]
    return x.lower().replace('- ',',').replace('&',',').replace('/',',').replace('.','').replace(', ',',').replace(' ,',',').replace(' and ',',').replace('el ','').replace('el-','').strip()

## Applying the function
job_posts_df['city_clean'] = job_posts_df.city.apply(lambda x : strip_city_name(x))   

In [60]:
# this part is hard coded however it may be done other way using 'Fuzzy Lookup'
# collect the words relative for each group

# abroad  in relative to Egypt
abroad = ['SHARJAH','Riyadh', 'travel jeddah' ,'الرياض', 'ابها', 'Riyadh, Saudi Arabia' ,'Riyadh, KSA','Dubai','Middle East','Global']


# remote work either online or from home
remote = ['Remote (Work from Home)','Remote','Remotely from home',  'manalonline' ,'Online From Home', 'Work from home']


# unspecified job posts
unknwon =['To be definied','outside cairo']


# west of Egypt
west_region =['North Coast','North Cost','Marsa Matrouh','Alamein','North Cost']


# Upper Eegypt and Red Sea areas
upper_redsea_regions =['Hurghada','ASWAN','Marsa Alam','qina','El Fayoum','Bani Suief','El Menya', 
                       'red sea','Sohag','el minya','Aswan/ Qena','Minya - Assiut - Sohag',
                       'Assuit-Menia-Mansoura-El Bhira-Sohag','Upper Egypt & Red Sea','Upper Egypt / Red Sea',
                       'Upper Egypt','الغردقة','Minia','Qena','ain sokhna','Red Sea/ Sinai',
                       'Elwadi Elgded','Owainat East','east oweinat','East Owinat','East Owainat','Owinat East',
                       'Asyut','Ein sokhna','al fayoum','El Menia','luxor','al-minya','Ain El Sokhna','sharm elsheikh',
                       'EL Minia','al ain al sokhna', 'Minya','Wadi El Notron','new valley','Assuit',
                       'Assiut','fayoum']


# sinai area
sinai_region =['Sharm Elkheikh','sinai','South sinai','sharm el shaik','Ariesh','Arish']


# Cario districts
cairo_regions =['kairo','great cairo','maadi','maadi,cairo','Maddi','nasr city' ,'Doki','Mohand','El Mohndseen','Ain Shams',
                'Dokki - Mohandseen','Mohandeseen','Heliopolis','obour' ,'Obour city','nozha' ,"Ma'adi", 'Obour','zamalek',
                'Abou Rawash','Misr- El Gedida','Helwan','5th Settelment','maady','El-Obour','elobour','EL Obour city',
                'elharam','new nozha','Dokki' ,'Ain Shams/ Helwan' , 'El Obour' , 'Mohandessin','Mokattam','alabassia',
                'Mohandiseen','Badr','حلوان','zahraa el maadi','التجمع الأول', 'Haram','Obuor city', 'El-shrouk','مدينة بدر',
                'new egypt','El Obour Industrial City','مدينه نصر', 'newegypt','15th of May','El-Sherouk'] 


In [61]:
def city_cleaner(k):
    "take k city name as a str and edit it to be as one of the unified categries of Areas as above mentioned lists "
    if k[0] =='c' or k[0] == 'C' and 'Canal' not in k or 'Cairo' in k or 'القاهر' in k or 'Down Town' in k or k in cairo_regions:
        k = 'Cairo'
    elif 'Alex' in k or 'alex' in k or 'الاسكندرية' in k:
        k ='Alexandria'
    elif '6' in k or 'October' in k or 'أكتوبر' in k or 'sheikh zayed' in k or 'Shiekh' in k or 'Sheikh' in k:
        k = 'October'
    elif 'G' in k and 'iza' in k or 'g' in k and 'iza' in k or 'الجيز' in k or k == 'Giaz' or 'GIZA' in k or 'Jizah' in k or k == 'gize':
        k = 'Giza'
    elif '10' in k :
        k = '10th Ramadan'
    elif 'All ' in k or 'all ' in k or 'Any ' in k or k == 'مصر' or k == 'all' or k =='Egype' or k =='Egypt' or 'anywhere' in k or 'al city' in k or 'Any' in k:
        k = 'All country'
    elif 'Isma' in k or 'Isam' in k or 'Port' in k or 'port' in k or 'Sue' in k or 'الإسماعيلية' in k or 'siuz' in k or 'Canal' in k:
        k = 'Canal'
    elif k in sinai_region:
        k = 'Sinai'
    elif k in west_region:
        k = 'West'
    elif k in upper_redsea_regions:
        k = 'Upper Egypt & Red Sea'
    elif k in abroad:
        k = 'Abroad'
    elif k in remote:
        k = 'Remotely'
    elif k in unknwon:
        k = 'other'
    else:
        k = 'Delta'
    return k

In [62]:
# fix the city column
job_posts_df.city_clean = job_posts_df.city_clean.apply(lambda x: city_cleaner(x))

# compare our results
display(job_posts_df.sample(5,random_state =7)[['city','city_clean']])

# check for the work
display(job_posts_df.city_clean.value_counts())

,city,city_clean
2182,Cairo,Cairo
1071,6th of October,October
6925,Tanta - Zagazig - Mansoura (Delta),Delta
17673,Cairo,Cairo
13141,Cairo,Cairo


Cairo                    15382
Giza                      3996
Alexandria                1084
Delta                      879
October                    228
10th Ramadan               156
All country                 62
Upper Egypt & Red Sea       32
Canal                       25
Abroad                       2
Sinai                        2
Remotely                     1
other                        1
Name: city_clean, dtype: int64

#####fix the career level column

In [63]:
# viewing unique values
job_posts_df.career_level.unique()

array(['Entry Level', 'Experienced (Non-Manager)', 'Manager',
       'Senior Management (e.g. VP, CEO)', 'Student'], dtype=object)

In [64]:
# fix the career_level
job_posts_df['career_level'] = job_posts_df['career_level'].apply(lambda x: x.split('(')[0])

# check
job_posts_df.career_level.unique()

array(['Entry Level', 'Experienced ', 'Manager', 'Senior Management ',
       'Student'], dtype=object)

##### Saving the cleaned data for further analysis

In [67]:
# save into csv 
applications_df.to_csv('clean_applications.csv', index=False)
job_posts_df.to_csv('clean_job_posts.csv', index=False)

In [68]:
applications_df.shape

(1854190, 4)

In [69]:
job_posts_df.shape

(21850, 19)